In [2]:
# Import our dependencies
# !pip install keras_tuner
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import keras_tuner as kt
import datetime
import shutil
import keras
import ast
import re


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("./DataFiles/HR_Analytics.csv")
final_results_all = []
application_df.head()

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,...,3,80,0,0,2,3,0,0,0,0.0
1,RM302,18,18-25,No,Travel_Rarely,812,Sales,10,3,Medical,...,1,80,0,0,2,3,0,0,0,0.0
2,RM458,18,18-25,Yes,Travel_Frequently,1306,Sales,5,3,Marketing,...,4,80,0,0,3,3,0,0,0,0.0
3,RM728,18,18-25,No,Non-Travel,287,Research & Development,5,2,Life Sciences,...,4,80,0,0,2,3,0,0,0,0.0
4,RM829,18,18-25,Yes,Non-Travel,247,Research & Development,8,1,Medical,...,4,80,0,0,0,3,0,0,0,0.0


In [4]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmpID                     1480 non-null   object 
 1   Age                       1480 non-null   int64  
 2   AgeGroup                  1480 non-null   object 
 3   Attrition                 1480 non-null   object 
 4   BusinessTravel            1480 non-null   object 
 5   DailyRate                 1480 non-null   int64  
 6   Department                1480 non-null   object 
 7   DistanceFromHome          1480 non-null   int64  
 8   Education                 1480 non-null   int64  
 9   EducationField            1480 non-null   object 
 10  EmployeeCount             1480 non-null   int64  
 11  EmployeeNumber            1480 non-null   int64  
 12  EnvironmentSatisfaction   1480 non-null   int64  
 13  Gender                    1480 non-null   object 
 14  HourlyRa

In [8]:
application_df.corr()

C:\Users\T852\AppData\Local\Temp\ipykernel_53500\1123807899.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  application_df.corr()


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,0.011584,-0.000329,0.205689,NaN,-0.011181,0.011617,0.022704,0.026804,0.509758,...,0.053632,NaN,0.038545,0.680837,-0.016819,-0.021704,0.310550,0.213330,0.216755,0.209987
DailyRate,0.011584,1.000000,-0.007134,-0.016750,NaN,-0.054996,0.019176,0.021349,0.043964,0.001940,...,0.012214,NaN,0.042097,0.015036,0.004297,-0.037683,-0.033737,0.010452,-0.032439,-0.022281
DistanceFromHome,-0.000329,-0.007134,1.000000,0.019661,NaN,0.038137,-0.013633,0.027899,0.006427,0.007360,...,0.002762,NaN,0.045153,0.007802,-0.035882,-0.030581,0.010111,0.018593,0.011817,0.010849
Education,0.205689,-0.016750,0.019661,1.000000,NaN,0.038020,-0.026831,0.019446,0.042908,0.097107,...,-0.005762,NaN,0.016717,0.145258,-0.026476,0.008806,0.065983,0.057557,0.051507,0.068713
EmployeeCount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EmployeeNumber,-0.011181,-0.054996,0.038137,0.038020,NaN,1.000000,0.021359,0.030896,-0.007231,-0.016891,...,-0.074316,NaN,0.057548,-0.013731,0.021525,0.009202,-0.010756,-0.008735,-0.011731,-0.009781
EnvironmentSatisfaction,0.011617,0.019176,-0.013633,-0.026831,NaN,0.021359,1.000000,-0.053826,-0.013254,0.000260,...,0.009256,NaN,0.002250,-0.002472,-0.016531,0.024867,-0.000318,0.017658,0.013731,0.003072
HourlyRate,0.022704,0.021349,0.027899,0.019446,NaN,0.030896,-0.053826,1.000000,0.046327,-0.028585,...,0.000071,NaN,0.049756,-0.004128,-0.010846,-0.004835,-0.018848,-0.023756,-0.026131,-0.003166
JobInvolvement,0.026804,0.043964,0.006427,0.042908,NaN,-0.007231,-0.013254,0.046327,1.000000,-0.013770,...,0.032878,NaN,0.022028,-0.007114,-0.018256,-0.008954,-0.022974,0.005862,-0.025929,0.037990
JobLevel,0.509758,0.001940,0.007360,0.097107,NaN,-0.016891,0.000260,-0.028585,-0.013770,1.000000,...,0.018063,NaN,0.015803,0.781880,-0.017370,0.038027,0.535210,0.390189,0.355518,0.371987


In [9]:
application_df.corr().to_csv('./DataFiles/hr_analysis_correlation.csv')

C:\Users\T852\AppData\Local\Temp\ipykernel_53500\3281731666.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  application_df.corr().to_csv('./DataFiles/hr_analysis_correlation.csv')


# Define Functions

In [22]:
# Delete old dirctory used by keras_tuner.Hyperband before a new keras_tuner.Hyperbad is made
def delete_directory_and_its_content(directory_name):

    # Use shutil.rmtree() to recursively delete directories and subdirectories
    try:
        shutil.rmtree(directory_name)
        print(f"Directory '{directory_name}' has been deleted successfully.")
    except OSError as e:
        print(f"Error: {e}")

In [23]:
# Creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()


    # number of layers including input layer but excluding output layer
    num_layers = hp.Int('num_layers', 1, 6)

    activation = hp.Choice('activation_layer_input', ['relu', 'tanh', 'leaky_relu'])

    #  Allow kerastuner to decide which activation function to use in hidden layers
    if activation == 'relu':
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('neurons_layer_input', min_value=X.shape[1], max_value=round(X.shape[1] * 2), step=5), activation='relu', input_dim=X.shape[1]))
    elif activation == 'tanh':
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('neurons_layer_input', min_value=X.shape[1], max_value=round(X.shape[1] * 2), step=5), activation='tanh', input_dim=X.shape[1]))
    elif activation == 'leaky_relu':
        activation = tf.keras.layers.LeakyReLU(alpha=0.01)
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('neurons_layer_input', min_value=X.shape[1], max_value=round(X.shape[1] * 2), step=5), activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_dim=X.shape[1]))


    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(1, num_layers):

        activation = hp.Choice(f'activation_layer_{i}', ['relu', 'leaky_relu'])

        #  Allow kerastuner to decide which activation function to use in hidden layers
        if activation == 'leaky_relu':
            activation = tf.keras.layers.LeakyReLU(alpha=0.01)
        nn_model.add(tf.keras.layers.Dense(units=hp.Int(f'neurons_layer_{i}',
            min_value=2,
            max_value=round(X.shape[1] * 2),
            step=1),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # hp.Choice('random_seed2', [str(np.random.get_state())])
    # Compile the model
    lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    nn_model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=lr), metrics = ["accuracy"])#optimizer='adam', metrics=["accuracy"])

    return nn_model

In [24]:
# Function to otherize values from multiple columns
def otherizations(df, list_of_columns_to_otherize, list_of_list_of_values_to_replace):
    for index in range(len(list_of_columns_to_otherize)):
        df = otherize_column(df, list_of_columns_to_otherize[index], list_of_list_of_values_to_replace[index])

    return df

In [25]:

# Function to change value of column to "other" (typically used for rare values)
def otherize_column(df, column, list_of_values_to_replace):
    df_copy = df.copy()


    # Replace in dataframe
    for app in list_of_values_to_replace:
        df_copy[column] = df_copy[column].replace(app,"Other")

    # Check to make sure binning was successful
    # print(df_copy[column].value_counts())

    return df_copy

In [26]:
# Remove outliers from a given column when a value's z-score for that column is greater than a given limit
def remove_outliers(df, threshold, columns_to_remove_outliers_from):
    # Calculate Z-scores only for specified columns
    z_scores = np.abs(stats.zscore(df[columns_to_remove_outliers_from]))

    # Create a mask for outliers
    outliers = (z_scores > threshold)

    # Create a DataFrame with outliers for further inspection if needed
    outlier_rows = df[df.index.isin(df[columns_to_remove_outliers_from].index[outliers.any(axis=1)])]

    # Remove rows with outliers in the specified columns
    df_cleaned = df[~df.index.isin(df[columns_to_remove_outliers_from].index[outliers.any(axis=1)])]

    print("Outlier rows:")
    print(outlier_rows.index)
    
    return outlier_rows, df_cleaned

In [27]:
# Remove outliers using the 1.5 InterQuartile Range Rule as the cuttoff values
def remove_outliers_1_5_IQR_Rule(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
    
        # Filter out rows outside of the IQR for this column
        df = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]

    return df

In [28]:
# Extract X and y from the given dataframe
def extract_X_y(df, y_column_name):
    # Split our preprocessed data into our features and target arrays
    # print(a2pplication_df.columns)
    X = df.drop(columns=[y_column_name])
    y = df[y_column_name]
    return (X, y)

In [29]:
# Create a specific model instead of rely on keras_tuner.hyperband
def create_specific_model(list_of_activations, num_layers, list_of_number_of_nodes_hidden_layer, input_dim, output_activation, filepath_to_model_weights):

    nn = tf.keras.models.Sequential()

    for layer_number in range(0, num_layers):
        if list_of_activations[layer_number] == 'leaky_relu':
            activation = tf.keras.layers.LeakyReLU(alpha=0.01)
        else:
            activation = list_of_activations[layer_number]
        if layer_number == 0:
            # Input layer which requires input_dim
            nn.add(tf.keras.layers.Dense(units = list_of_number_of_nodes_hidden_layer[layer_number], activation = activation, input_dim = input_dim))
        else:
            nn.add(tf.keras.layers.Dense(units = list_of_number_of_nodes_hidden_layer[layer_number], activation = activation))


    # Output layer
    nn.add(tf.keras.layers.Dense(units = 1, activation = output_activation))
    
    if (filepath_to_model_weights != ''):
        nn.load_weights(filepath_to_model_weights)

    # Check the structure of the model
    print(nn.summary())
    return nn

In [30]:
# Find and return X_train_scaled and X_test_scaled
def scale_it(X_train, X_test):
    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

In [31]:
# Find and return X_train_scaled and X_test_scaled
def scale_it(X_train, X_test):
    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

In [32]:
# Expose Random Seed of Tensorflow
class RandomSeedLogger(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        # Access the TensorFlow random seed and log it
        random_seed = tf.random.get_seed()
        print(f"Random Seed: {random_seed}")

# Create an instance of the custom callback
random_seed_logger = RandomSeedLogger()

In [33]:
# Otherize not value explicitly but by knowing the ordinal popularity of the value (e.g. '11' for 11th most common value)
def otherizer2(df, column, starting_from_least_column):
    to_replace = [df[column].value_counts().index[starting_from_least_column:]]
    # print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
    print(f'len(to_replace): {len(to_replace[0].tolist())}')
    # print(f'to_replace: {to_replace}')
    # Replace in dataframe
    for cls in to_replace:
        df[column] = df[column].replace(cls,"Other")

    # print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
    return df

In [34]:
# Optimization by otherizing across a range of values
def optimization2(optimization_title, df, columns, start, finish, step, reset_df):
    if reset_df:
        df = application_df.drop(columns=['EIN', 'NAME'])
    for column in columns:
        if len(df[column].value_counts()) > 5:
            print(f'start: {start}')
            for starting_from_least_column in range(start, finish, step):
                if reset_df:
                    df = application_df.drop(columns=['EIN', 'NAME'])

                print(f'start: {start}')
                print(f'starting_from_least_column: {starting_from_least_column}')
                print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
                df = otherizer2(df, column, starting_from_least_column)
                print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
                run_hyperparameters(optimization_title + f': otherize columns from {starting_from_least_column}', df, 8, 9, '', 20)

In [35]:
# Return the last n lines of a file
def read_last_n_lines(file_name, n):
    
    with open(file_name, 'r') as file:
        lines = file.readlines()
        return lines[-n:]

In [59]:
# Create hyperband models for multiple z-score thresholds
def run_hyperparameters(optimization_title, df_new, columns_to_remove_outliers_from, begin_threshold, end_threshold, column_to_remove, max_epochs, restrict_columns_to_interquartile):
    global X, y, final_results_all, X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled

    if column_to_remove in df_new.columns:
        df_new_removed_column = df_new.drop(columns=[column_to_remove])
    else:
        df_new_removed_column = df_new


    df_new_removed_column = pd.get_dummies(df_new_removed_column)
    # return df_new_removed_column
    # print('number of columns', len(df_new_removed_column.columns))
    # return
    for z_threshold in range(begin_threshold, end_threshold):

        np.random.seed(42)
        tf.random.set_seed(42)
        number_of_rows_before_outliers_removed = df_new_removed_column.shape[0]

        if restrict_columns_to_interquartile:
            no_outliers_df = remove_outliers_1_5_IQR_Rule(df_new_removed_column)
        else:
            outlier_rows, no_outliers_df = remove_outliers(df_new_removed_column, z_threshold, columns_to_remove_outliers_from)

        number_of_outlier_rows = number_of_rows_before_outliers_removed - no_outliers_df.shape[0]
        


        file_path = "./DataFiles/optimization_results.txt"
        # past_results_records = read_last_n_lines(file_path, 2)
        # prev_number_of_outlier_rows = re.search("'number of outlier rows': (\d+)", past_results_records[1]).group(1)
        # print(prev_number_of_outlier_rows)
        # if int(prev_number_of_outlier_rows) == number_of_outlier_rows:
        #     # Skip to the next threshold
        #     # continue
        #     pass
        # print('number of columns', len(df_new_removed_column.columns))
        # return

        # Split our preprocessed data into our features and target array
        X, y = extract_X_y(no_outliers_df, 'Attrition')

        # Split the preprocessed data into a training and testing dataset
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            random_state=3,
                                                            stratify=y)

        # Standardize X_train and X_test
        X_train_scaled, X_test_scaled = scale_it(X_train, X_test)

        delete_directory_and_its_content('./untitled_project/')



        tuner = kt.Hyperband(
            create_model,
            objective="val_accuracy",
            max_epochs=max_epochs,
            hyperband_iterations=2,
            seed=1)

        # Run the kerastuner search for best hyperparameters
        tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))
        first_model = tuner.get_best_models(num_models=1)[0]
        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        model_loss, model_accuracy = first_model.evaluate(X_test_scaled,y_test,verbose=2)
        first_model.save_weights(f'./DataFiles/model_weights_{timestamp}_{float(model_accuracy*100):.2f}.h5')
        dict_results = {}
        dict_results['model_accuracy'] = model_accuracy
        dict_results['z_threshold'] = z_threshold
        
        dict_results['Optimization Title'] = optimization_title
        dict_results['number of columns'] = len(df_new_removed_column.columns)
        dict_results['param'] = tuner.get_best_hyperparameters(1)[0].values
        dict_results['columns'] = df_new_removed_column.columns
        dict_results['removed column'] = column_to_remove
        dict_results['number of outlier rows'] = number_of_outlier_rows
        dict_results['number of rows'] = no_outliers_df.shape[0]
        dict_results['max_epoch'] = max_epochs
        dict_results['hyperband seed'] = 1
        dict_results['np.rand.seed and tf.random.set_seed'] = 42
        final_results_all.append(dict_results)

        file_path = "./DataFiles/optimization_results.txt"

        # Open the file in append mode
        with open(file_path, "a") as file:
            # Append a line to the file
            # line_to_append = "This is a new line to append to the file."
            file.write(str(dict_results) + "\n")
    print(final_results_all)

# Preprocessing: Remove Non-Beneficial Columns

In [52]:
# Create a dictionary with column names and their unique value counts
unique_counts = {col:application_df[col].nunique() for col in application_df.columns}

# Convert the dictionary to a DataFrame
unique_counts_df = pd.DataFrame(list(unique_counts.items()), columns=['Column Name', 'Unique Values'])

# Display the new DataFrame
print(unique_counts_df)


                 Column Name  Unique Values
0                      EmpID           1470
1                        Age             43
2                   AgeGroup              5
3                  Attrition              2
4             BusinessTravel              4
5                  DailyRate            886
6                 Department              3
7           DistanceFromHome             29
8                  Education              5
9             EducationField              6
10             EmployeeCount              1
11            EmployeeNumber           1470
12   EnvironmentSatisfaction              4
13                    Gender              2
14                HourlyRate             71
15            JobInvolvement              4
16                  JobLevel              5
17                   JobRole              9
18           JobSatisfaction              4
19             MaritalStatus              3
20             MonthlyIncome           1349
21                SalarySlab    

In [53]:
a1pplication_df = application_df.copy()

print(a1pplication_df.shape)

(1480, 38)


In [54]:


# Identify columns with only one unique value
cols_to_drop = [col for col in a1pplication_df.columns if a1pplication_df[col].nunique() == 1]

# Drop these columns from the DataFrame
a1pplication_df.drop(cols_to_drop, axis=1, inplace=True)
a1pplication_df.shape


(1480, 35)

In [62]:
# Remove id column
a1pplication_df.drop(['EmpID'], axis=1, inplace=True)
a1pplication_df.shape

(1480, 34)

# Preprocessing: Convert 2 valued columns to '1's and '0's

In [63]:
a1pplication_df['Attrition'].unique()

array(['Yes', 'No'], dtype=object)

In [64]:
a2pplication_df = a1pplication_df.copy()
print(a2pplication_df.shape)
print(a2pplication_df.info())
a2pplication_df.head()

(1480, 34)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1480 non-null   int64  
 1   AgeGroup                  1480 non-null   object 
 2   Attrition                 1480 non-null   object 
 3   BusinessTravel            1480 non-null   object 
 4   DailyRate                 1480 non-null   int64  
 5   Department                1480 non-null   object 
 6   DistanceFromHome          1480 non-null   int64  
 7   Education                 1480 non-null   int64  
 8   EducationField            1480 non-null   object 
 9   EmployeeNumber            1480 non-null   int64  
 10  EnvironmentSatisfaction   1480 non-null   int64  
 11  Gender                    1480 non-null   object 
 12  HourlyRate                1480 non-null   int64  
 13  JobInvolvement            1480 non-null   int64  
 1

,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,405,...,3,3,0,0,2,3,0,0,0,0.0
1,18,18-25,No,Travel_Rarely,812,Sales,10,3,Medical,411,...,3,1,0,0,2,3,0,0,0,0.0
2,18,18-25,Yes,Travel_Frequently,1306,Sales,5,3,Marketing,614,...,3,4,0,0,3,3,0,0,0,0.0
3,18,18-25,No,Non-Travel,287,Research & Development,5,2,Life Sciences,1012,...,3,4,0,0,2,3,0,0,0,0.0
4,18,18-25,Yes,Non-Travel,247,Research & Development,8,1,Medical,1156,...,3,4,0,0,0,3,0,0,0,0.0


In [65]:
a2pplication_df['Attrition'] = a2pplication_df['Attrition'].replace('No',"0")
a2pplication_df['Attrition'] = a2pplication_df['Attrition'].replace('Yes',"1")
a2pplication_df['Attrition'] = a2pplication_df['Attrition'].astype('int64')
print(a2pplication_df.shape)
print(a2pplication_df.info())
a2pplication_df.head()

(1480, 34)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1480 non-null   int64  
 1   AgeGroup                  1480 non-null   object 
 2   Attrition                 1480 non-null   int64  
 3   BusinessTravel            1480 non-null   object 
 4   DailyRate                 1480 non-null   int64  
 5   Department                1480 non-null   object 
 6   DistanceFromHome          1480 non-null   int64  
 7   Education                 1480 non-null   int64  
 8   EducationField            1480 non-null   object 
 9   EmployeeNumber            1480 non-null   int64  
 10  EnvironmentSatisfaction   1480 non-null   int64  
 11  Gender                    1480 non-null   object 
 12  HourlyRate                1480 non-null   int64  
 13  JobInvolvement            1480 non-null   int64  
 1

,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,18,18-25,1,Travel_Rarely,230,Research & Development,3,3,Life Sciences,405,...,3,3,0,0,2,3,0,0,0,0.0
1,18,18-25,0,Travel_Rarely,812,Sales,10,3,Medical,411,...,3,1,0,0,2,3,0,0,0,0.0
2,18,18-25,1,Travel_Frequently,1306,Sales,5,3,Marketing,614,...,3,4,0,0,3,3,0,0,0,0.0
3,18,18-25,0,Non-Travel,287,Research & Development,5,2,Life Sciences,1012,...,3,4,0,0,2,3,0,0,0,0.0
4,18,18-25,1,Non-Travel,247,Research & Development,8,1,Medical,1156,...,3,4,0,0,0,3,0,0,0,0.0


# Create Model

In [66]:
start_threshold = 100
end_threshold = 101
max_epoch = 10

columns = ['Age']
for column in columns:
    df = a2pplication_df.copy()
    restrict_columns_to_interquartile = False
    columns_to_remove_outliers_from = []
    columns_to_remove_outliers_from.append(column)
    run_hyperparameters(f'"{columns_to_remove_outliers_from}" is the only column with outliers removed; try hyperparameters across thresholds {start_threshold} to {end_threshold} excluding {end_threshold} with max_epoch {max_epoch}', df, columns_to_remove_outliers_from, start_threshold, end_threshold, '', max_epoch, restrict_columns_to_interquartile)

Trial 60 Complete [00h 00m 02s]
val_accuracy: 0.8405405282974243

Best val_accuracy So Far: 0.8405405282974243
Total elapsed time: 00h 01m 50s
12/12 - 0s - loss: nan - accuracy: 0.8405 - 125ms/epoch - 10ms/step
[{'model_accuracy': 0.8405405282974243, 'z_threshold': 100, 'Optimization Title': '"[\'Age\']" is the only column with outliers removed; try hyperparameters across thresholds 100 to 101 excluding 101 with max_epoch 10', 'number of columns': 1533, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 1897, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel',
       ...
       'JobRole_Sales Representative', 'MaritalStatus_Divorced',
       'MaritalStatus_Married', 'MaritalStatus_Single', 'Sa

# Reproduce Results of Model Found By Hyperband

## Here are the specs from the best fitting model

#### (These specifications along many other optimization tests were captured in the file ./DataFiles/optimization_results.txt)<br><br>

{'model_accuracy': 0.9645830988883972, <br>

In [67]:
readme


The accuracy for the model was X which was above 75% by Q%. 
The accuracy fpr 

SyntaxError: invalid syntax (3850848940.py, line 4)